In [88]:
import os
import subprocess
import Bio
from Bio import SeqIO 
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
import glob
from Bio import SeqRecord
import pandas as pd
import genomicranges as gr
from Bio.Seq import Seq
import pyranges as pr
from Bio import AlignIO, SeqIO

In [2]:
def make_sbatch_file(filename):
    
    my_list = ["#!/bin/bash", 
               "#SBATCH --job-name=all_bz", 
               "#SBATCH --nodes=1",  
               "#SBATCH --ntasks=1",                     
               "#SBATCH --cpus-per-task=10",              
               "#SBATCH --mem=20gb",                    
               "#SBATCH --partition=20",                
               "##SBATCH --output all_bz-%j.out",  
               "#SBATCH --mail-type=ALL",               
               "#SBATCH --mail-user=hlharris@wi.mit.edu"] 
    
    with open(filename, "w") as file: 
        for item in my_list:
            file.write(item + '\n') 
    
def calc_zeros(alignment): 
    count = 0 
    return [0 if base == "-" else (count := count + 1) for base in alignment[0]]


def crop_alignment(start, end, alignment):
        
    zeros_seq = calc_zeros(alignment)
    #print(zeros_seq) #MIGHT NEED TO SUBTRACT 1 FROM EACH 
    try:
        ix_start = zeros_seq.index(start)
        #print(ix_start)
        ix_end = zeros_seq.index(end) #pretty elegant
        #print(ix_end)
        cropped_alignment = alignment[:, ix_start:ix_end]
        return cropped_alignment
    except: 
        return None
    
    
def concat_alignment(gtf_file, alignment):
   # print(alignment)
    complete_align_type = MultipleSeqAlignment([]) 
    #add the groups to complete align: 
    for recordix in range(len(alignment)): 
        #new_record = SeqRecord.SeqRecord("") 
        new_record = SeqRecord("")
        new_record.id = alignment[recordix].id 
        new_record.seq = Seq("") #added this 

        complete_align_type.append(new_record)
    
   # print(complete_align_type)
    for index, row in gtf_file.iterrows():     
        crop_align = crop_alignment(row[3], row[4], alignment)
        #print(crop_align)
        if crop_align is not None: 
            for recordix in range(len(crop_align)): 
                complete_align_type[recordix].seq += crop_align[recordix].seq #append additional sequence
        
    return complete_align_type

In [8]:
genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

for gene in genes: 
    os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/') 
    subprocess.run(['mkdir', gene],  stderr=subprocess.PIPE) 
    
dicty = SeqIO.to_dict(SeqIO.parse('/lab/page_scratch/hannah/pileatedgibbon/' + "genomic", format = "fasta"))


NameError: name 'animal' is not defined

In [13]:
dicty = SeqIO.to_dict(SeqIO.parse('/lab/page_scratch/hannah/pileatedgibbon/' + "genomic", format = "fasta")) 

fasta_file = dicty['LG04']     
SeqIO.write(fasta_file, '/lab/page_scratch/hannah/pileatedgibbon/LGO4.fa', 'fasta') 

fasta_file = dicty['chrY_decollapsed']     
SeqIO.write(fasta_file, '/lab/page_scratch/hannah/pileatedgibbon/chrY.fa', 'fasta') 

fasta_file = dicty['chrX']     
SeqIO.write(fasta_file, '/lab/page_scratch/hannah/pileatedgibbon/chrX.fa', 'fasta')

1

In [80]:
x_genes = ["EIF1AX",  "KDM5C", "KDM6A", "ZFX",  "DDX3X",  "USP9X",  "RPS4X"] 
genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 
species = ["gorilla",  "orangutan", "pileatedgibbon"]  


seq_df = pd.read_csv('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_6sp/sequence_gorilla_orang.csv', delimiter="\t" ) 

for animal in species: 
    
    directory = "/lab/page_scratch/hannah/" + animal + "/" 
    for gene in genes: 
        print(animal, gene) 

        seq_df1 = seq_df[seq_df['gene_name'] == gene] 
        seq_df2 = seq_df1[seq_df1['animalz'] == animal]
       
        start = int(seq_df2['start']) 
        
        end =   int(seq_df2['end']) 
        print(start, end) 
        
        if gene in x_genes:
            fasta_file = SeqIO.read(directory + "chrX.fa", "fasta") 
       #deleted line here 
        else: 
            
            fasta_file = SeqIO.read(directory + "chrY.fa", "fasta")
        
        fasta_seq = fasta_file.seq
        
        os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + gene)
               
        sliced_sequence = fasta_seq[start:end]
        records = [SeqIO.SeqRecord(sliced_sequence, id=animal)] 
        SeqIO.write(records, animal, 'fasta') 
        

gorilla NLGN4X
16331058 16695004
177553137
363946
gorilla NLGN4Y
29825722 30468514
open chry
65269191
642792
gorilla TXLNG
27276084 27346285
177553137
70201
gorilla TXLNGY
27803965 27833965
open chry
65269191
30000
gorilla EIF1AX
30609319 30635804
177553137
26485
gorilla EIF1AY
27086783 27112110
open chry
65269191
25327
gorilla KDM5D
27649979 27699687
open chry
65269191
49708
gorilla KDM5C
63332398 63385367
177553137
52969
gorilla UTY
31188944 31459416
open chry
65269191
270472
gorilla KDM6A
55257862 55523882
177553137
266020
gorilla ZFY
9548537 9606261
open chry
65269191
57724
gorilla ZFX
34654573 34729921
177553137
75348
gorilla DDX3Y
31753158 31780594
open chry
65269191
27436
gorilla DDX3X
51703161 51745464
177553137
42303
gorilla USP9Y
31940705 32122962
open chry
65269191
182257
gorilla USP9X
51453340 51618157
177553137
164817
gorilla RPS4Y1
9402733 9487480
open chry
65269191
84747
gorilla RPS4X
83999000 84012235
177553137
13235
gorilla TMSB4X
23447702 23457789
177553137
10087
gori

In [74]:
#take the region alignments, and remove any region with gaps_rename *filtered*

In [8]:
#repeat mask all files for each species masking primate repeats (hard mask) 
#resubmit for alignment - rename everything with 'masked' 

genes = ['NLGN4X', 'NLGN4Y', "TXLNG", "TXLNGY","EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y"] 
species = ["human"] #, "chimp", "mac", "marm","gorilla",  "orangutan", "pileatedgibbon", "lemur", "loris"]  

os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/')

with open("repeat_mask_commands1.py", "a") as file: 
    file.write('import os' + ' \n')
    file.write('import subprocess' + ' \n') 

    for gene in genes:
     
        file.write('os.chdir("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + gene + '")'  + ' \n')
        
        for specy in species:
            
            file.write('subprocess.run(["RepeatMasker", "-noint", "-species", "primates", "' + specy + '"''],  stderr=subprocess.PIPE)' + ' \n')
            
            
make_sbatch_file("run_masker1.sh") 
print('here')
with open("run_masker1.sh", "a") as file: 
    print('here')
    file.write('python repeat_mask_commands1.py') 



here
here


In [15]:
#after masking step is done -- repeat alignments with everything labelled 'masked'
#this might not actually work

genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 
print(len(genes)) 
#genes = ["KDM5C", "TMSB4Y", "ZFY", "EIF1AX", "TXLNGY"] 

genes = ['NLGN4X']
species_dict = {0:"humanmasked", 1:"chimp", 2: "mac", 3: "marm"} 

files_generated = [] 

for gene in genes:

    records = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/' + gene + ".fa", "fasta"))
    
    
    #set the trees for input 
    if len(records) == 3: 
        tree = '((((((humanmasked chimp) gorilla) orangutan) pileatedgibbon) mac) lemur loris)'
    else: 
        tree = '((((((humanmasked chimp) gorilla) orangutan) pileatedgibbon) mac marm) lemur loris)'
    
    #make a new directory for each gene: 
    os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/'+ gene)

    
    subprocess.run(["cp", "human.masked", "humanmasked"])

    subprocess.run(["touch", "all_bz_masked.log"])  #making a new file 
    
    with open("all_bz_masked.log", "w") as log_file:
        result = subprocess.run(['all_bz', '-', tree], stdout=log_file, stderr=subprocess.PIPE, check=True)
        
    make_sbatch_file("testj1_masked.sh") #make sbatch file - it makes a new one each time through so you dont have to delete the old one 
    print(gene) 
    with open("testj1_masked.sh", "a") as file: #append to the file 
        file.write("bash ") 
        file.write("all_bz_masked.log" + "\n")
        file.write("tba '" + tree + "' *.*.maf tba.maf >&tba.log" + "\n") 
        file.write("maf_project tba.maf humanmasked '" + tree + "' > humanmasked_proj.maf" + "\n") 
        file.write("msa_view -o FASTA humanmasked_proj.maf > " + gene + "_msamasked.fa")
        
    subprocess.run(["sbatch", 'testj1_masked.sh'], stderr=subprocess.PIPE)
    
    files_generated = [] 
    os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/')  

20
NLGN4X
Submitted batch job 501588


In [1]:
##REAL RUN

In [89]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

genes = [ 'NLGN4X', 'NLGN4Y', "TXLNG", "TXLNGY","EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y"] 
print(len(genes)) 

genes = ["USP9X"]
#genes = ["EIF1AX"] 

regions = ["upmax", "downmax", "promoter", "max_peak", "upstream", "exon", "intron"]  
#genes =['NLGN4X'] 
#genes = ['DDX3X'] 
species_dict = {0:"human", 1:"chimp", 2: "mac", 3: "marm"} 

files_generated = [] 
spec1 = 'humanmasked'
#genes = [  "RPS4Y1"] 
#regions = ["promoter"]  



for gene in genes: 
    print(gene)
    records = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/' + gene + ".fa", "fasta")) 
    
    #set the trees for input 
    if len(records) == 3: 
        tree = '((((((humanmasked chimp) gorilla) orangutan) pileatedgibbon) mac) lemur loris)'
        list_of_species = ['chimp', 'gorilla', 'orangutan', 'pileatedgibbon', 'mac', 'lemur', 'loris']
    else: 
        tree = '((((((humanmasked chimp) gorilla) orangutan) pileatedgibbon) mac marm) lemur loris)' 
        list_of_species = ['chimp', 'gorilla', 'orangutan', 'pileatedgibbon', 'mac', 'marm', 'lemur', 'loris']

        
    #make a new directory for each gene: 
    os.chdir('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + gene)
    
    
    gtf_file = pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + gene + "_gtf_all100923.txt", delimiter="\t", header = None) #pd.read_csv("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/" + gene + "_gtf_all0721.txt", delimiter="\t", header = None) 
    #print(gtf_file)
    
    for spec2ix in range(len(list_of_species)): 
       # print(spec2ix, 'im at the top')
        
        spec2 = list_of_species[spec2ix]
        spec2ix = spec2ix + 1
        second_align_file_name = spec1 + "." + spec2  
        
       # output = subprocess.run(["msa_view", "-o", "FASTA", second_align_file_name + ".sing.maf"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)  #making a new file 
        msa = list(SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + gene + "/" + gene + "_msamasked.fa", "fasta")) #get the MASKED version there
      #  print(msa)
        new_filepath = "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/" + gene + "/" + second_align_file_name + "_msa.fa"
        pair_alignment = MultipleSeqAlignment([msa[0]]) #add human alignment

        pair_alignment.append(msa[spec2ix]) 
       # print(pair_alignment, "HI")

        SeqIO.write(pair_alignment, new_filepath, "fasta") #as fasta

        for region in regions: 
         

            gtf_file1 = gtf_file[gtf_file[2] == region] 
           
            new_alignment = AlignIO.read("/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/" + gene + "/" + second_align_file_name + "_msa.fa", "fasta")
            returned_align = concat_alignment(gtf_file1,new_alignment) 
            filtered_sequences = [record for record in returned_align if has_ACGT(record.seq)] 
            if filtered_sequences:
                SeqIO.write(filtered_sequences, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/" + gene + "/" + second_align_file_name + "_" + region + "_msa.phy", "phylip") #as phylip
                
                new_alignment = MultipleSeqAlignment([])

                for record in filtered_sequences:
    # Replace '*' with '-' in the sequence
                    modified_seq = str(record.seq).replace('*', '-')
    # Replace 'N' with '-'
                    modified_seq = modified_seq.replace('N', '-')
    
    # Create a new SeqRecord with the modified sequence
                    new_record = SeqRecord(Seq(modified_seq), id=record.id)
    
    # Append the new record to the new alignment
                    new_alignment.append(new_record)
        
                SeqIO.write(new_alignment, "/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/" + gene + "/" + second_align_file_name + "_" + region + "_msa.phy", "phylip") #as phylip

                subprocess.run(['/lab/page_scratch/hannah/trimal/source/trimal', '-in', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + gene + '/' + second_align_file_name + '_' + region + '_msa.phy', '-out', '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/multiz_7sp/' + gene + '/' + second_align_file_name + '_' + region + '_msa_filtered.phy', '-gt', '1'],  stderr=subprocess.PIPE)

            else:  
                print("check") 

20
USP9X
now im here and UGH
upmax region
now im here and UGH
downmax region
now im here and UGH
promoter region
now im here and UGH
max_peak region
now im here and UGH
upstream region
now im here and UGH
exon region
now im here and UGH
intron region
now im here and UGH
upmax region
now im here and UGH
downmax region
now im here and UGH
promoter region
now im here and UGH
max_peak region
now im here and UGH
upstream region
now im here and UGH
exon region
now im here and UGH
intron region
now im here and UGH
upmax region
now im here and UGH
downmax region
now im here and UGH
promoter region
now im here and UGH
max_peak region
now im here and UGH
upstream region
now im here and UGH
exon region
now im here and UGH
intron region
now im here and UGH
upmax region
now im here and UGH
downmax region
now im here and UGH
promoter region
now im here and UGH
max_peak region
now im here and UGH
upstream region
now im here and UGH
exon region
now im here and UGH
intron region
now im here and UGH
upm